<div class="alert alert-block alert-info">
LINMA2415 - Project - Question 18

- Colson Adeline 19752000
    
- Lhernould Benoit 77102200 

- El Bared Mary Jo 48212000 
  
    
</div>

<div class="alert alert-block alert-info">
Load Data
</div>

In [1]:
using CSV, DataFrames, Dates, JuMP, Statistics, Plots, Gurobi

In [2]:
df = CSV.read("data.csv", DataFrame)
D_t = df[!, "FR_load_forecast_entsoe_transparency"]

println(size(D_t))
println(first(D_t, 5))

df = CSV.read("data.csv", DataFrame)
D_t = df[!, "FR_load_forecast_entsoe_transparency"]

println(size(D_t))
println(first(D_t, 5))
        
file_path = "capacity_pv.csv"
df = CSV.File(file_path; header=true) |> DataFrame
capacity_pv = Array(df)
println(first(capacity_pv, 10))
    
file_path = "capacity_wind.csv"
df = CSV.File(file_path; header=true) |> DataFrame
capacity_wind = Array(df)
println(first(capacity_wind, 10))

(8760,)
[56250.0, 54300.0, 53600.0, 50000.0, 47100.0]
(8760,)
[56250.0, 54300.0, 53600.0, 50000.0, 47100.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.001, 0.067, 0.218]
[0.12594, 0.123693, 0.124232, 0.13045, 0.140045, 0.136766, 0.129185, 0.135791, 0.152821, 0.14136]


<div class="alert alert-block alert-info">
Loading and filtering imbalance data
</div>

In [3]:
file_path = "Imbalance.csv"
data = CSV.read(file_path, DataFrame)

# The column containing the target data
imbalance_column_name = "Total Imbalance [MWh] - SCA|FR"

# Check if the column is present
if !(imbalance_column_name in names(data))
    error("Column '$imbalance_column_name' not found in the data.")
end

# Extract the "Total Imbalance [MWh] - SCA|FR" data
imbalance_data = data[!, imbalance_column_name]

# Convert non-numeric values to missing values
function to_int(value)
    try
        return parse(Int, value)
    catch
        return missing
    end
end

# Apply the conversion function and filter out missing values
imbalance = filter(!ismissing, to_int.(imbalance_data))
sorted_data = sort(vec(sum(reshape(imbalance, 2, :), dims=1)))

8606-element Vector{Union{Missing, Int64}}:
   15
   17
   19
   22
   22
   22
   23
   25
   25
   25
   26
   26
   29
    ⋮
 3539
 3585
 3715
 3723
 3737
 3797
 3814
 3820
 3923
 4421
 4543
 5197

<div class="alert alert-block alert-info">
Question 17 - Computing ORDC Curve
</div>

In [4]:
function compute_lolp(reserve, sorted_imbalance)
    # Probability of an imbalance higher than the given reserve
    return mean(sorted_imbalance .> reserve)
end


discretization_step = 20
reserve_range = minimum(sorted_data):discretization_step:maximum(sorted_data)

lolp_values = [compute_lolp(R, sorted_data) for R in reserve_range]

VOLL = 5000

ordc_values = lolp_values .* VOLL

# Plot the ORDC curve
# plot(reserve_range, ordc_values, label="ORDC", color=:blue, xlabel="Reserve (MW)", ylabel="ORDC Value (\$)", title="Operating Reserve Demand Curve (ORDC)", legend=:topright, grid=true)

sorted_data = sort(vec(sum(reshape(imbalance, 2, :), dims=1)))

8606-element Vector{Union{Missing, Int64}}:
   15
   17
   19
   22
   22
   22
   23
   25
   25
   25
   26
   26
   29
    ⋮
 3539
 3585
 3715
 3723
 3737
 3797
 3814
 3820
 3923
 4421
 4543
 5197

<div class="alert alert-block alert-info">
Computing utility for each R_t using rectangle method
</div>

In [5]:
reserve_range_shift = [R - minimum(sorted_data) for R in reserve_range]
reserve_size = length(reserve_range_shift)

U_ORDC= Float64[]
for R in reserve_range_shift
    # Define the interval for the current reserve R
    intervals = collect(0:discretization_step:R)
    
    # Calculate the area under the curve for the current reserve R using rectangles method
    area = 0.0
    for i in 1:length(intervals)-1
        height = ordc_values[findfirst(x -> x ≥ intervals[i], reserve_range_shift)]
        width = intervals[i+1] - intervals[i]
        area += height * width
    end
    push!(U_ORDC, area)
end

<div class="alert alert-block alert-info">
Constants and data
</div>

In [6]:
# Constants and Data
T = 8760
lambda = 5000  
cost_of_debt = 0.04
cost_of_equity = 0.07
corporate_tax = 0.30
economic_life = 20
carbon_tax = 50 

capacity = Dict(
    "Coal" => 1,
    "CCGT" => 1,
    "OCGT" => 1,
    "Onshore Wind" => capacity_wind,
    "Offshore Wind" => capacity_wind,
    "PV" => capacity_pv
)

# Technologies data
technologies = Dict(
    "Coal" => (capex=2000*1000, om=0.03, debt_ratio=0.62, heat_rate=2.4, EA=175.2252157*1000, price =5, C=12, emissions=1.4),
    "CCGT" => (capex=950*1000, om=0.03, debt_ratio=0.56, heat_rate=1.62, EA=85.60445144*1000, price =30, C=48.6, emissions=0.5),
    "OCGT" => (capex=700*1000, om=0.03, debt_ratio=0.6, heat_rate=2.5, EA=61.91153839*1000, price =30, C=75, emissions=0.6),
    "Onshore Wind" => (capex=700*1000, om=0.03, debt_ratio=0.7, heat_rate=0, EA=58.99797384*1000, price =0, C=0, emissions=0.0),
    "Offshore Wind" => (capex=1300*1000, om=0.03, debt_ratio=0.7, heat_rate=0, EA=109.5676657*1000, price =0, C=0, emissions=0.0),
    "PV" => (capex=400*1000, om=0.03, debt_ratio=0.8, heat_rate=0, EA=32.04823387*1000, price =0, C=0, emissions=0.0)
)

technologies_t = Dict(
    "Coal" => (capex=2000*1000, om=0.03, debt_ratio=0.62, heat_rate=2.4, EA=175.2252157*1000, price =5, C=12, emissions=1.4),
    "CCGT" => (capex=950*1000, om=0.03, debt_ratio=0.56, heat_rate=1.62, EA=85.60445144*1000, price =30, C=48.6, emissions=0.5),
    "OCGT" => (capex=700*1000, om=0.03, debt_ratio=0.6, heat_rate=2.5, EA=61.91153839*1000, price =30, C=75, emissions=0.6)
)

technologies_r = Dict(
    "Onshore Wind" => (capex=700*1000, om=0.03, debt_ratio=0.7, heat_rate=0, EA=58.99797384*1000, price =0, C=0, emissions=0.0),
    "Offshore Wind" => (capex=1300*1000, om=0.03, debt_ratio=0.7, heat_rate=0, EA=109.5676657*1000, price =0, C=0, emissions=0.0),
    "PV" => (capex=400*1000, om=0.03, debt_ratio=0.8, heat_rate=0, EA=32.04823387*1000, price =0, C=0, emissions=0.0)
)

#Battery data
F_b=6*1000
A_b=1.620627574*1000
eta=0.9
capex_b= 25000

25000

<div class="alert alert-block alert-info">
Question 18
</div>

In [7]:
T = 8760
model = Model(Gurobi.Optimizer)


@variable(model, k[g in keys(technologies)] >= 0)  # Installed capacity
@variable(model, p[g in keys(technologies), t=1:T] >= 0)  # Hourly production
@variable(model, r[g in keys(technologies_t), t=1:T] >= 0)  # Hourly reserve # seulement pour les thermiques
# @variable(model, R[t=1:T]>=0) #Total Hourly Reserve
@variable(model, z[t=1:T, i=1:reserve_size], Bin)  # Binary variable for reserve selection
@variable(model, d[t=1:T] >=0)
#For Battery
@variable(model, k_b>=0 ) 
@variable(model, e[1:T] >= 0)
@variable(model, p_in[1:T] >=0)
@variable(model, p_out[1:T] >=0)



# Define the objective function
@objective(model, Min,
    sum((technologies[g].om * technologies[g].capex + technologies[g].EA) * k[g] +
        sum(technologies[g].C * p[g, t] for t in 1:T) for g in keys(technologies)) +
    (A_b + F_b) * k_b -
    sum(lambda * d[t] for t in 1:T) +
    carbon_tax * sum(technologies[g].emissions * sum(p[g, t] for t in 1:T) for g in keys(technologies)) -
    sum(U_ORDC[i] * z[t, i] for t in 1:T, i = 1:reserve_size)
)
    
@constraint(model, [t in 1:T], d[t]<=D_t[t])
@constraint(model, price[t in 1:T], -d[t] + p_out[t] - p_in[t]  + sum( p[g,t] for g in keys(technologies))==0)

@constraint(model, [t in 1:T], e[t]<= 4*k_b)
@constraint(model, [t in 1:T], p_in[t]<=k_b)
@constraint(model, [t in 1:T], p_out[t]<=k_b)
@constraint(model,  e[1]==0)
@constraint(model, [t in 2:T], e[t]== e[t-1] + 1*(sqrt(eta)* p_in[t-1] - 1/sqrt(eta) * p_out[t-1]))

for g in ["Coal", "CCGT", "OCGT"]
    @constraint(model, [t in 1:T], p[g, t] + r[g, t] <= k[g])
end
for t in 1:T
    @constraint(model, p["Onshore Wind", t] <= capacity_wind[t]*k["Onshore Wind"])
    @constraint(model, p["Offshore Wind", t] <= capacity_wind[t]*k["Offshore Wind"])
    @constraint(model, p["PV", t] <= capacity_pv[t]*k["PV"])
end

#reserve constraints
@constraint(model, [t in 1:T], sum(r[g,t] for g in keys(technologies_t))==sum(reserve_range_shift[i] * z[t, i] for i in 1:reserve_size))
# Ensure exactly one `z[t, i]` is 1 for each `t`
@constraint(model, [t = 1:T], sum(z[t, i] for i = 1:reserve_size) == 1)



optimize!(model)

print("Variables value, k:",value.(k))
print("Variables value, k_b:",value.(k_b))
# print("Variables value, d:",value.(d))
print("Objective value: ", objective_value(model))
print(keys(technologies))

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-31
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 122640 rows, 2391487 columns and 4875211 nonzeros
Model fingerprint: 0x9f70f676
Variable types: 113887 continuous, 2277600 integer (2277600 binary)
Coefficient statistics:
  Matrix range     [1e-03, 5e+03]
  Objective range  [7e+01, 4e+06]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+05]
Found heuristic solution: objective 0.0649962
Presolve removed 12868 rows and 12867 columns (presolve time = 5s) ...
Presolve removed 12869 rows and 12868 columns (presolve time = 10s) ...
Presolve removed 12869 rows and 12868 columns (presolve time = 15s) ...
Presolve removed 12869 rows and 12868 columns (presolve time = 20s) ...
Presolve removed 12869 ro

<div class="alert alert-block alert-info">
Results
</div>

In [8]:
println("\n")
println("Point I")
println("Optimal capacities (MW):")
total_production = 0
production_by_tech = Dict(g => 0.0 for g in keys(technologies))
for g in keys(technologies)
    cap = value(k[g])  # This ensures 'cap' remains a floating-point number
    println("$(g): $(cap) MW")
    for t in 1:T
        production_by_tech[g] += value(p[g, t])  # Accumulate as floating-point values
        total_production += value(p[g, t])  # Keep total production as floating-point
    end
end
    cap = value(k_b)  # This ensures 'cap' remains a floating-point number
    println("Battery: $(cap) MW")

println("\n")

println("Point II")
println("Share of each technology in effective production:")
for g in keys(technologies)
    share = production_by_tech[g] / total_production
    println("$(g): $(share * 100) %")
end
println("\n")


println("Point III")
println("Total Production: $(total_production) MWh")
total_cost = capex_b*value(k_b) + sum((technologies[g].EA+technologies[g].om * technologies[g].capex)*value.(k[g])+sum((technologies[g].C)*value.(p[g,i]) for i in 1:T)  for g in keys(technologies))
println("Total Investment and Production Cost: €$(total_cost)")
println("\n")



println("Point IV")
# CO2 Emissions data
emissions = Dict(
    "Coal" => 1.4,
    "CCGT" => 0.5,
    "OCGT" => 0.6,
    "Onshore Wind" => 0.0,
    "Offshore Wind" => 0.0,
    "PV" => 0.0
)

# Calculating CO2 emissionys
total_co2_emissions = sum(emissions[g] * production_by_tech[g] for g in keys(production_by_tech))
println("\nTotal CO2 Emissions: $(total_co2_emissions) tons")
println("\n")


println("Point V")

load_curtailment = sum(max(0, D_t[t] - value(d[t])) for t in 1:T)
println("Total Load Curtailment: $(load_curtailment) MWh")
println("\n")


# println("Point VI")
# prices = [-JuMP.dual(price[i]) for i in 1:T]

# for g in keys(technologies)
#     profit[g] = -(technologies[g].EA + technologies[g].capex) * value(k[g]) - 
#                  sum((technologies[g].C - prices[i]) * value(p[g, i]) for i in 1:T)
# end

# for g in keys(technologies)
#     println("Profit for technology $g: $(profit[g])")
# end



Point I
Optimal capacities (MW):
Coal: 0.0 MW
CCGT: 22285.026096206024 MW
Offshore Wind: 0.0 MW
OCGT: 13186.93738339172 MW
PV: 116744.72730847783 MW
Onshore Wind: 151923.93313616575 MW
Battery: 186833.65132862423 MW


Point II
Share of each technology in effective production:
Coal: 0.0 %
CCGT: 10.087037900210715 %
Offshore Wind: 0.0 %
OCGT: 1.239984194052471 %
PV: 30.893887448693835 %
Onshore Wind: 57.77909045704235 %


Point III
Total Production: 4.796018705195526e8 MWh
Total Investment and Production Cost: €2.840019361183284e10


Point IV

Total CO2 Emissions: 2.7757003658006866e7 tons


Point V
Total Load Curtailment: 2043.7654290730643 MWh


